In [1]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

/home/tristan/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tristan/miniconda3/lib/python3.8/site-packages/fastbook/__init__.py:11: UserWarning: Missing `ipywidgets` - please install it
  except ModuleNotFoundError: warn("Missing `ipywidgets` - please install it")
/home/tristan/miniconda3/lib/python3.8/site-packages/fastbook/__init__.py:13: UserWarning: Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`
  except ModuleNotFoundError: warn("Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`")


In [2]:
path = untar_data(URLs.MNIST)

In [3]:
Path.BASE_PATH = path

In [4]:
path.ls()

(#2) [Path('training'),Path('testing')]

In [5]:
def img_to_df(img_path):
    img = Image.open(img_path)
    img_t = tensor(img)
    df = pd.DataFrame(img_t)
    display(df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys'))

In [6]:
def stack_imgs(path):
    images = path.ls().sorted()
    tensors = [tensor(Image.open(i)) for i in images]
    stacked_tensors = torch.stack(tensors).float()/255
    print(path.stem, stacked_tensors.shape)
    return stacked_tensors

In [7]:
list_digit_tensors = [stack_imgs(i) for i in (path/'training').ls().sorted()]

0 torch.Size([5923, 28, 28])
1 torch.Size([6742, 28, 28])
2 torch.Size([5958, 28, 28])
3 torch.Size([6131, 28, 28])
4 torch.Size([5842, 28, 28])
5 torch.Size([5421, 28, 28])
6 torch.Size([5918, 28, 28])
7 torch.Size([6265, 28, 28])
8 torch.Size([5851, 28, 28])
9 torch.Size([5949, 28, 28])


In [8]:
def get_digit_means(list_digit_tensors):
    means = []
    for i, digit_t in enumerate(list_digit_tensors):
        means.append(digit_t.mean(0))
    return means

In [9]:
digit_means = get_digit_means(list_digit_tensors)

In [10]:
def predict_digit(digit_t, mean_digits):
    pred = None
    lowest_dist = None
    for i, mean_digit in enumerate(mean_digits):
        distance = (digit_t - mean_digit).abs().mean((-1,-2))
        if pred is None:
            pred = i
            lowest_dist = distance
        if max(lowest_dist, distance) is lowest_dist:
            pred = i
            lowest_dist = distance

    return pred

In [11]:
def naive_classifier(paths, digit_means):
    accuracy_table = {}
    for i, path in enumerate(paths):
        preds = []
        
        for tensor in path:
            pred = predict_digit(tensor, digit_means)
            preds.append(pred == i)
            
        accuracy = torch.Tensor(preds).float().mean()
        accuracy_table[i] = accuracy

    return sum(accuracy_table.values()) / len(accuracy_table.values())

In [12]:
accuracy = naive_classifier(list_digit_tensors, digit_means)
accuracy

tensor(0.6429)

In [13]:
(path).ls()

(#2) [Path('training'),Path('testing')]

In [14]:
train_x = torch.cat([stack_imgs((path/'training/8')), stack_imgs((path/'training/9'))]).view(-1, 28*28)
train_y = tensor([1]*len((path/'training/8').ls().sorted()) + [0]*len((path/'training/9').ls().sorted())).unsqueeze(1)
dset = list(zip(train_x,train_y))

8 torch.Size([5851, 28, 28])
9 torch.Size([5949, 28, 28])


In [15]:
valid_x = torch.cat([stack_imgs((path/'testing/8')), stack_imgs((path/'testing/9'))]).view(-1, 28*28)
valid_y = tensor([1]*len((path/'training/8').ls().sorted()) + [0]*len((path/'training/9').ls().sorted())).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

8 torch.Size([974, 28, 28])
9 torch.Size([1009, 28, 28])


In [16]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [17]:
def sigmoid(x): return 1/(1+torch.exp(-x))

In [18]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [19]:
# Init Weights
weights = init_params((28*28,1))
bias = init_params(1)

In [20]:
dl = DataLoader(dset, batch_size=256)
xb,yb = first(dl)

In [21]:
valid_dl = DataLoader(valid_dset, batch_size=256)

In [22]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

In [23]:
def train_epoch(model, lr, params):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

In [24]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [25]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [26]:
class BasicOptim:
    def __init__(self,params,lr): self.params,self.lr = list(params),lr

    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr

    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad = None

In [27]:
def train_epoch(model):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

In [28]:
dls = DataLoaders(dl, valid_dl)

In [29]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

In [30]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [31]:
learn.fit(40, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.436656,0.885521,0.000000,00:00
1,0.247001,0.944375,0.000504,00:00
2,0.148541,0.869253,0.084216,00:00
3,0.093255,0.779283,0.219869,00:00
4,0.065683,0.712776,0.296520,00:00
5,0.051776,0.667641,0.344932,00:00
6,0.044290,0.636992,0.377711,00:00
7,0.039877,0.615711,0.396873,00:00
8,0.037015,0.600436,0.410489,00:00
9,0.034998,0.589049,0.421079,00:00
